<a href="https://colab.research.google.com/github/fabriciocarraro/Google-ADK_AI-Agent-for-social-media/blob/main/Google_ADK_AI_Agent_for_social_media.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q google-genai google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

In [4]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)

In [5]:
# Cria um serviço de sessão em memória

session_service = InMemorySessionService()

In [6]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final

async def call_agent(agent: Agent, message_text: str) -> str:
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = await session_service.create_session(app_name=agent.name, user_id="user1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    async for event in runner.run_async(user_id="user1", session_id=session.id, new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [7]:
# Lista todos os modelos disponíveis atualmente

for model in client.models.list():
    print(model.name)

models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-pro-preview-05-06
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview

In [8]:
MODELO_RAPIDO = "gemini-2.0-flash"
MODELO_ROBUSTO = "gemini-2.5-pro-preview-03-25"

In [9]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################

async def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model=MODELO_RAPIDO,
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    # Executa o agente
    lancamentos = await call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [10]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################

async def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model=MODELO_RAPIDO,
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais.
        Você recebe uma lista de lançamentos recentes do agente buscador e,
        - Para cada um dos lançamentos recebidos, você deve usar a ferramenta
        de busca do Google (google_search) para buscar os pontos mais relevantes
        que poderíamos abordar em um post sobre cada um deles.
        Você também pode usar o (google_search) para encontrar mais
        informações sobre cada um dos temas e aprofundar.
        - Depois de terminar a busca, você irá escolher APENAS UM tema dentre todos eles,
        aquele tema com potencial de ser o mais relevante com base nas suas pesquisas.
        - Depois de escolher o tema mais relevante dentre todos eles, você irá retornar
        qual foi o tema escolhido, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"

    # Executa o agente
    plano_do_post = await call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [11]:
######################################
# --- Agente 3: Redator do Post --- #
######################################

async def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model=MODELO_ROBUSTO,
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"

    # Executa o agente
    post_final = await call_agent(redator, entrada_do_agente_redator)
    return post_final

In [12]:
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [13]:
from datetime import date

data_de_hoje = date.today().strftime("%d/%m/%Y")

In [16]:
print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 3 Agentes 🚀\n")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes
if not topico:
    print("\nVocê esqueceu de digitar o tópico!")
else:
    print(f"\nMaravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = await agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = await agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    post_redigido = await agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(post_redigido))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 3 Agentes 🚀

❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: lançamentos do evento Google I/O

Maravilha! Vamos então criar o post sobre novidades em lançamentos do evento Google I/O

--- 📝 Resultado do Agente 1 (Buscador) ---



> Para encontrar os lançamentos mais relevantes do Google I/O, farei algumas buscas no Google para identificar os anúncios que geraram mais notícias e entusiasmo no último mês.
> 
> 
> Com base nas notícias do Google I/O 2025, aqui estão os lançamentos e anúncios mais relevantes, focados em IA e suas aplicações:
> 
> 1.  **Gemini 2.5 Pro e Flash:** A nova geração dos modelos de IA do Google, com melhorias significativas em raciocínio, velocidade e custo-eficiência. O Gemini 2.5 Pro se destaca como o modelo mais inteligente do Google até o momento, enquanto o Gemini 2.5 Flash é otimizado para tarefas rápidas e econômicas. Ambos estão sendo integrados em diversos produtos, como o aplicativo Gemini, Google Cloud, Chrome e Android.
> 
> 2.  **AI Mode no Google Search:** Uma nova forma de usar o Google Search, permitindo conversas mais complexas e respostas geradas por IA. O "Modo IA" (AI Mode) está sendo disponibilizado nos Estados Unidos e promete entregar resultados mais elaborados, com resumos e comparações gerados a partir de múltiplas fontes. O Google também apresentou o "Deep Search" para pesquisas ainda mais aprofundadas.
> 
> 3.  **Project Astra:** Uma visão de assistente universal, capaz de entender o mundo ao seu redor e interagir de forma contextual. O Project Astra usa o Gemini para compreender vídeo em tempo real, compartilhar tela com contexto, ter memória personalizada e controlar o computador por voz ou texto. Esses recursos estão sendo integrados ao Gemini Live e a novos dispositivos, como óculos com IA.
> 
> 4.  **Veo 3:** Modelo de IA para geração de vídeos com som nativo e efeitos realistas. O Veo 3 permite criar vídeos a partir de prompts de texto, com sincronização labial, ruídos de fundo contextuais e diálogos integrados. Essa ferramenta está disponível para assinantes do plano Ultra e integrada ao Gemini app e à nova ferramenta Flow.
> 
> 5.  **Android XR e Project Moohan:** A plataforma Android XR para dispositivos de realidade aumentada e o Project Moohan, um headset desenvolvido em parceria com a Samsung e a Qualcomm. O Android XR promete revolucionar a interação com o mundo digital, enquanto o Project Moohan visa transformar videoconferências em experiências 3D imersivas, utilizando IA para tradução em tempo real e percepção de contexto.
> 
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Okay, com base nos lançamentos do Google I/O 2025 que você me forneceu, vou pesquisar mais informações sobre cada um deles para identificar qual tem o maior potencial para um post de mídia social.
> 
> 
> Após analisar os lançamentos do Google I/O 2025, o tema que considero mais relevante para um post de mídia social é o **AI Mode no Google Search**.
> 
> **Justificativa:**
> 
> *   **Amplo alcance:** O Google Search é usado por bilhões de pessoas diariamente. A introdução do AI Mode tem o potencial de mudar fundamentalmente a forma como as pessoas interagem com a informação online.
> *   **Novidade:** O lançamento do AI Mode para todos nos Estados Unidos é uma notícia recente e de grande impacto.
> *   **Discussão:** O tema gera discussões sobre o futuro da busca, o papel da IA na organização da informação e o impacto nos criadores de conteúdo.
> *   **Recursos:** O AI Mode oferece novas funcionalidades, como conversas mais complexas, respostas geradas por IA e o "Deep Search" para pesquisas aprofundadas.
> 
> **Plano para o Post:**
> 
> 1.  **Título:** Google Search turbinado por IA: Descubra o AI Mode e o futuro da busca!
> 2.  **Introdução:**
>     *   O que é o AI Mode e por que ele é importante.
>     *   Como o AI Mode muda a forma de pesquisar no Google.
>     *   O contexto do lançamento no Google I/O 2025.
> 3.  **Principais recursos do AI Mode:**
>     *   Conversas complexas com o Google Search.
>     *   Respostas geradas por IA a partir de múltiplas fontes.
>     *   O que é o "Deep Search" e como ele aprofunda as pesquisas.
>     *   Integração com o Gemini para funcionalidades como o "Personal Context" e sugestões personalizadas.
> 4.  **Impacto e discussão:**
>     *   Como o AI Mode pode afetar a forma como as pessoas aprendem e encontram informação.
>     *   O impacto para criadores de conteúdo e SEO.
>     *   Privacidade e uso de dados pessoais pelo AI Mode.
> 5.  **Chamada para ação:**
>     *   Incentivar os usuários a experimentar o AI Mode (se disponível em sua região).
>     *   Convidar os seguidores a compartilhar suas opiniões e experiências com a nova ferramenta.
> 
> **Possíveis formatos:**
> 
> *   Post de texto com imagens e GIFs explicativos.
> *   Vídeo curto demonstrando o AI Mode em ação.
> *   Enquete para saber a opinião dos seguidores sobre o futuro da busca com IA.
> 
> Este plano oferece uma abordagem completa e informativa sobre o AI Mode no Google Search, explorando seus recursos, impacto e potencial para o futuro da busca online.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> Okay, Alura! Preparei um post caprichado sobre o AI Mode no Google Search, direto do forno do Google I/O 2025. A ideia é deixar a galera curiosa e mostrar como a busca que a gente conhece está evoluindo! 🚀
> 
> ---
> 
> ✨ **O Google Search NUNCA MAIS SERÁ O MESMO!** ✨
> 
> E aí, dev & curioso por tecnologia! 👋 Você piscou e o Google I/O 2025 chegou com uma bomba: o **AI Mode no Google Search**! 🤯 Sabe aquela busca tradicional? Esquece! Estamos falando de uma revolução na forma como encontramos informação.
> 
> Mas calma, o que é esse tal de AI Mode? 🤔
> É o Google Search turbinado com Inteligência Artificial, permitindo:
> 
> *   🗣️ **Conversas complexas:** Faça perguntas elaboradas como se estivesse batendo um papo com um expert. Chega de palavras-chave soltas!
> *   🧠 **Respostas geradas por IA:** Em vez de só links, receba resumos inteligentes e completos, compilados de diversas fontes.
> *   🔎 **"Deep Search":** Quer mergulhar de cabeça em um assunto? Essa função te leva para as profundezas do conhecimento, organizando informações complexas de forma clara.
> *   🔮 **Integração com Gemini:** Espere por contexto pessoal e sugestões personalizadas que entendem VOCÊ!
> 
> **O impacto? Gigante!** 💥
> Isso muda como aprendemos, como criamos conteúdo e até o SEO que conhecemos. A IA está reorganizando o conhecimento mundial, e na Alura, a gente AMA ver a tecnologia transformando o futuro!
> 
> E aí, preparado(a) para essa nova era da busca? O AI Mode já está sendo lançado nos EUA! 🇺🇸
> 
> 👇 Conta pra gente: qual sua expectativa para essa novidade? Como você acha que o AI Mode vai impactar seu dia a dia e seus estudos?
> 
> #GoogleIO #InteligenciaArtificial #FuturoDaBusca #AluraOnline


--------------------------------------------------------------
